> 시작에 앞서...  
> Colab을 [연결]한 뒤 [런타임] 탭에서 [런타임 유형 변경]을 TPU 또는 GPU로 바꿔 주세요.

**Tensorflow 2.0 에서 실습하는 float16 모델 생성 및 학습**
=================

#Overview
1. Tensorflow2.0 에서 floating point precision을 결정하는 [dtype policy()](https://www.tensorflow.org/api_docs/python/tf/keras/mixed_precision/experimental/Policy)에 대해 이해합니다.
2. float16/mixed_float16/mixed_bfloat16 방식으로 모델을 생성하고 학습해 봅니다.
3. loss scale을 이해하고, 위의 Policy에서 자동으로 설정되지 않은 loss scale을 직접 설정해 봅니다.



In [0]:
import tensorflow as tf
tf.__version__

'2.2.0-rc2'

# 16bit floating point를 사용하는 이유

> _python의 기본적인 dtype 및 연산 방법은 모두 float32에 최적화 되어 있고, 정확도도 낮은 16bit를 굳이 왜 ...?_  

1. **학습 시간의 단축**
  - 16bit floating point 연산에 최적화 된 연산장치(Tensor Core)를 갖고 있는 경우 학습 시간의 단축 효과를 볼 수 있다.
    - RTX line-up, Titan-V, V100, TPU
  - (오해 no) 입력 데이터를 16bit로 준비할 필요는 없다. 오히려 numpy 등 연산 처리에서 비효율적.

2. **모델 크기의 축소**
  - 연산 자원이 부족한 edge computing device에서 인공신경망 모델을 활용하기 위함
    - Ex) DCASE2020 Task1b는 모델 크기에 500KB 용량 제한을 둠 (float32 기준 125k params)
  - 활용 bit 수를 절반으로 줄일 경우, 같은 메모리에 두 배 큰 노드 수를 가진 네트워크를 활용할 수 있게 됨.

# 실습 준비

In [0]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision
! nvidia-smi

Thu Apr 16 00:46:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Setting the dtype policy

*dtype policy*는 아래 항목들에 대해 작용하는 global policy.  
- 모델을 구성하는 각 layer들의 dtype
- ('mixed_float16'의 경우) optimizer의 DynamicLossScaling

In [0]:
policy = mixed_precision.Policy('float16')
mixed_precision.set_policy(policy)

*dtype policy*는 layer를 구성하는 아래 두 가지 요소에 작용한다.
- Compute dtype
  - 연산 정확도에 대한 dtype
  - (활용) float16으로 설정하여 연산 속도에서 이득을 가져갈 수 있다.
- Variable dtype
  - Layer를 구성하는 variable에 대한 dtype
  - (활용) float16으로 설정하여 모델 용량에 대해 이득을 가져갈 수 있다.

In [0]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float16


설정할 수 있는 dtype police를 알아보면,
- 'float32' (기본값)
  - compute_dtype = float32
  - variable_dtype = float32
- 'float16' / 'bfloat16'
  - compute_dtype = float16
  - variable_dtype = float16
- 'mixed_float16' / 'mixed_bfloat16'
  - compute_dtype = float16
  - variable_dtype = float32

In [0]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


# Building a toy model

2개의 fully-connected(Dense) layer로 구성된 간단한 모델을 만들어 본다.

In [0]:
inputs = keras.Input(shape=(784,), name='digits')
num_units = 4096

dense1 = layers.Dense(num_units, activation='relu', name='dense_1')
x = dense1(inputs)
dense2 = layers.Dense(num_units, activation='relu', name='dense_2')
x = dense2(x)

모델의 각 layer는 dtype에 대한 policy를 가지며, 기본적으로 global dtype policy를 따른다.

즉, set_policy('mixed_float16')에서는
1. 각 layer의 variable은 float32
2. 모델에 입력되는 데이터는 float32 (첫 번째 layer에서 float16으로 cast)
2. 각 layer의 출력은 float16

으로 표현된다.

In [0]:
# 1. print variable dtype of dense1
print('dense1.kernel.dtype: %s' % dense1.kernel.dtype.name)
# 2. print dtype of input data
print('inputs.dtype: %s' % inputs.dtype.name)
# 3. print dtype of output data
print('x.dtype: %s' % x.dtype.name)


dense1.kernel.dtype: float32
inputs.dtype: float32
x.dtype: float16


> *Q. set_policy('float16')에서는 1.2.3. 의 dtype이 어떻게 될까요?*

## 출력 계층을 추가

- 출력 Activation layer는 *'float32'*로 override할 것
  - Activation layer에는 variable이 없기 때문

> *Normally, you can create the output predictions as INCORRECT one, but this is not always numerically stable with float16.*

In [0]:
# INCORRECT: softmax and model output will be float16, when it should be float32
# outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

# CORRECT: softmax and model output are float32
x = layers.Dense(10, name='dense_logits')(x)
outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

Outputs dtype: float32


(FYI) 모델의 마지막 계층이 Activation layer가 아닌 경우, 아래와 같이 모델의 출력을 float32로 cast해 줄 수 있다.

In [0]:
# The linear activation is an identity function. So this simply casts 'outputs'
# to float32. In this particular case, 'outputs' is already float32 so this is a
# no-op.
"""outputs = layers.Activation('linear', dtype='float32')(outputs)"""

# Loss scaling

'float16' 연산은 'float32'연산보다 dynamic range가 좁기 때문에 연산 과정에서 overflow/underflow가 발생하기 쉽다.  
따라서 아래와 같이 backpropagation 과정에서 충분히 값이 전달될 수 있도록 적당히 큰 값을 곱해주는 loss scaling 과정이 필요하다.

```
loss_scale = 1024
loss = model(inputs)
loss *= loss_scale
# We assume `grads` are float32. We do not want to divide float16 gradients
grads = compute_gradient(loss, model.trainable_variables)
grads /= loss_scale
```

'mixed_float16'에 대해서는 적당한 loss scale 값을 찾아주는 [DynamicLossScale](https://www.tensorflow.org/api_docs/python/tf/mixed_precision/experimental/DynamicLossScale)이 기본 loss scale로 설정된다.

In [0]:
loss_scale = policy.loss_scale
print('Loss scale: %s' % loss_scale)

Loss scale: DynamicLossScale(current_loss_scale=32768.0, num_good_steps=0, initial_loss_scale=32768.0, increment_period=2000, multiplier=2.0)


**'float16'에 대해서는 loss scale이 기본 적용 되어 있지 않다.**  
이에 float16 연산에 최적화 되지 않은 일부 optimizer(ex. Adam)들은 학습이 제대로 이루어지지 않기에, 아래와 같이 직접 loss_scale='dynamic'을 지정해 줄 필요가 있다.

In [0]:
new_policy = mixed_precision.Policy('float16')
print('Loss scale: %s' % new_policy.loss_scale)
new_policy = mixed_precision.Policy('float16', loss_scale='dynamic')
print('Forcing dynamic loss scale: %s' % new_policy.loss_scale)

Loss scale: None
Forcing dynamic loss scale: DynamicLossScale(current_loss_scale=32768.0, num_good_steps=0, initial_loss_scale=32768.0, increment_period=2000, multiplier=2.0)


혹은 다음과 같이 optimizer 자체에 loss scaling을 적용하는 [LossScaleOptimizer](https://www.tensorflow.org/api_docs/python/tf/keras/mixed_precision/experimental/LossScaleOptimizer) 기능이 있다.

In [0]:
# optimizer = keras.optimizers.RMSprop()
# optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')

"optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')"

# Let's Train model


In [0]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
history = model.fit(x_train, y_train,
                    batch_size=8192,
                    epochs=5,
                    validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])


Epoch 1/5
6/6 [==============================] - 1s 166ms/step - loss: 4.7684 - accuracy: 0.4044 - val_loss: 0.5788 - val_accuracy: 0.8455
Epoch 2/5
6/6 [==============================] - 1s 130ms/step - loss: 0.6886 - accuracy: 0.7854 - val_loss: 0.3394 - val_accuracy: 0.8898
Epoch 3/5
6/6 [==============================] - 1s 127ms/step - loss: 0.3419 - accuracy: 0.8927 - val_loss: 0.2997 - val_accuracy: 0.9062
Epoch 4/5
6/6 [==============================] - 1s 127ms/step - loss: 0.3276 - accuracy: 0.8989 - val_loss: 0.1920 - val_accuracy: 0.9353
Epoch 5/5
6/6 [==============================] - 1s 131ms/step - loss: 0.2535 - accuracy: 0.9175 - val_loss: 0.1628 - val_accuracy: 0.9475
313/313 - 1s - loss: 0.1776 - accuracy: 0.9443
Test loss: 0.17762431502342224
Test accuracy: 0.9442999958992004


# GPU performance tips

1. Batch size를 두 배로 키우세요
  - float16 tensor를 사용할 경우, 메모리의 절반만 필요로 하기 때문에 모델 성능에 영향이 없다면 batch size를 키워서 학습 속도를 향상
2. Tensor Core들을 활용할 수 있는 모델 구조를 만드세요
  - 최신 NVIDIA GPU에 탑재된 Tensor core들은 float16의 matrix multiplication을 매우 빠르게 처리할 수 있음
  - Tensor core들은 8의 배수로 구성된 tensor에서 동작하므로, 아래와 같은 모델 구조를 추천함
```
tf.keras.layers.Dense(units=64)
# for Conv2d/Conv3d etc.
tf.keras.layers.Conv2d(filters=48, kernel_size=7, stride=3)
# for RNN layers
tf.keras.layers.LSTM(units=64)
tf.keras.Model.fit(epochs=2, batch_size=128)
```



# *References*
https://www.tensorflow.org/guide/keras/mixed_precision